In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython import display
import glob
import pickle
import ipywidgets
import math
import json
%matplotlib inline

In [2]:

class Camera:
    def __init__(self):
        self.calibrated = False
        
    def calibrate(self, calibration_images, show_annoted = False, show_corrected = False):
        w=9
        h=6
        pattern_size = (w,h)

        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        single_object_points = np.zeros((w*h,3), np.float32)
        single_object_points[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)

        all_object_points = []
        all_image_points = []
        im_shape = None

        for image_path in calibration_images:
            im = plt.imread(image_path)
            im_gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
            im_gray = (im_gray*(np.max(im_gray)-np.min(im_gray))*255 ).astype('uint8')
            if im_shape == None:
                im_shape = im_gray.shape[::-1]
            if im_gray.shape[::-1] != im_shape:
                raise( ValueError('image different shape: '+image_path))
            found,corners = cv2.findChessboardCorners(im_gray,(9,6),None)
            if found==False: 
                continue
            corners2 = cv2.cornerSubPix(im_gray,corners,(11,11),(-1,-1),criteria)
            all_image_points.append(corners2)
            all_object_points.append(single_object_points)
            if show_annoted:
                plt.figure()
                im_annoted = cv2.drawChessboardCorners(im, pattern_size, corners2, found)
                plt.imshow(im_annoted)

        ret, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(all_object_points, all_image_points, im_shape,None,None)
        self.dist_coefs = dist_coefs
        self.camera_matrix = camera_matrix
        self.calibrated = True
        if show_corrected:
            for image_path in calibration_images:
                im = plt.imread(image_path)   
                plt.figure()
                plt.imshow(cv2.undistort(im,mtx,dst))
    
    def save_calibration(self, path=None):
        if path == None:
            path = 'camera_cal.pickle'
        cal = {}
        cal['dist_coefs'] = self.dist_coefs
        cal['camera_matrix'] = self.camera_matrix
        with open( path, "wb" )  as f:
            pickle.dump(cal,f)

    def load_calibration(self, path=None):
        if path == None:
            path = 'camera_cal.pickle'
        cal = {}
        try:
            with open( path, "rb" )  as f:
                cal = pickle.load(f)
                self.dist_coefs = cal['dist_coefs']
                self.camera_matrix = cal['camera_matrix']
                return True
        except:
            pass
        return False
        
            
    
    def undistort(self, im):
        if not self.calibrated:
            raise(ValueError('camera must first be calibrated'))
        return cv2.undistort(im,self.camera_matrix,self.dist_coefs)

In [7]:
#run them all
chessboard_paths = glob.glob('chessboard_images/mono/*')
all_calibrations = dict()
for chessboard_path in sorted(chessboard_paths):
    camera_name = chessboard_path.split('/')[-1]
    camera = Camera()
    print(chessboard_path)
    camera.calibrate(calibration_images = glob.glob(chessboard_path+'/*.png'))
    cal = dict()
    cal['dist_coefs'] = camera.dist_coefs.tolist()
    cal['camera_matrix'] = camera.camera_matrix.tolist()
    all_calibrations[camera_name] = cal
print(json.dumps(all_calibrations, indent=1))


chessboard_images/mono/elp1_left_1280_720
chessboard_images/mono/elp1_left_320_240
chessboard_images/mono/elp1_left_640_480
chessboard_images/mono/elp1_left_800_600
chessboard_images/mono/elp1_right_1280_720
chessboard_images/mono/elp1_right_320_240
chessboard_images/mono/elp1_right_640_480
chessboard_images/mono/elp1_right_800_600
{
 "elp1_left_800_600": {
  "dist_coefs": [
   [
    -0.446086481871062,
    0.23312642629504782,
    0.0006529640791213134,
    -0.0010675879541717482,
    -0.06825077051922872
   ]
  ],
  "camera_matrix": [
   [
    595.3696455677525,
    0.0,
    421.71191245012795
   ],
   [
    0.0,
    595.227838369773,
    300.7718037393661
   ],
   [
    0.0,
    0.0,
    1.0
   ]
  ]
 },
 "elp1_right_640_480": {
  "dist_coefs": [
   [
    -0.45965863446920213,
    0.27874645672261184,
    2.3786602812977865e-05,
    0.0005112069329298304,
    -0.10979625981449942
   ]
  ],
  "camera_matrix": [
   [
    600.980164676157,
    0.0,
    320.6794324601403
   ],
   [
    

In [14]:
chessboard_paths = sorted(glob.glob('chessboard_images/stereo/*'))
all_calibrations = dict()
for chessboard_path in sorted(chessboard_paths):
    camera_name = chessboard_path.split('/')[-1]
    for resolution_path in glob.glob(chessboard_path + "/*"):
        resolution_name = resolution_path.split('/')[-1]
        print('processing',camera_name,resolution_name)
        camera = Camera()

        camera.calibrate(calibration_images = glob.glob(resolution_path+'/*left.png'))
        cal = dict()
        cal['dist_coefs'] = camera.dist_coefs.tolist()
        cal['camera_matrix'] = camera.camera_matrix.tolist()
        all_calibrations[camera_name+'_left_'+resolution_name] = cal

        camera.calibrate(calibration_images = glob.glob(resolution_path+'/*right.png'))
        cal = dict()
        cal['dist_coefs'] = camera.dist_coefs.tolist()
        cal['camera_matrix'] = camera.camera_matrix.tolist()
        all_calibrations[camera_name+'_right_'+resolution_name] = cal

print(json.dumps(all_calibrations, indent=1))

processing elp1 1280_720
processing elp1 640_480
processing elp1 320_240
processing elp1 800_600
processing elp2 1280_720
processing elp2 640_480
processing elp2 320_240
processing elp2 800_600
{
 "elp2_right_320_240": {
  "dist_coefs": [
   [
    -0.24291046566044389,
    -0.12893886369524282,
    0.0032431485546723093,
    -0.006849764534526603,
    0.13152028120272452
   ]
  ],
  "camera_matrix": [
   [
    198.60499013459375,
    0.0,
    172.55854992900754
   ],
   [
    0.0,
    196.52463216145273,
    124.02403285751764
   ],
   [
    0.0,
    0.0,
    1.0
   ]
  ]
 },
 "elp1_left_320_240": {
  "dist_coefs": [
   [
    -0.44081686398279796,
    0.2850205126361244,
    -0.01378285490807083,
    0.00030593704239362924,
    -0.15753807161921618
   ]
  ],
  "camera_matrix": [
   [
    236.79060377016359,
    0.0,
    159.3647564686138
   ],
   [
    0.0,
    237.05319081154485,
    130.4899433621213
   ],
   [
    0.0,
    0.0,
    1.0
   ]
  ]
 },
 "elp1_right_800_600": {
  "dist_c